In [47]:
import pandas as pd

# Read in the data
crop_country = pd.read_csv('..\data-clean\\N_use_crop_country.csv')

crop_country.head()

,country,iso3_code,region_ifa,year,crop,crop_area_k_ha,n_k_t,aver_n_rate_kg_ha,fertilizer_emission_id,ghg_type_id,reliability_score,academic_citation,created_at
0,Argentina,ARG,Latin America,2018,Fruit primary and citrus fruits and treenuts,450.000,24.082438,53.516528,NaN,NaN,NaN,"Ludemann, C.I., Gruere, A., Heffer, P. et al. ...",NaN
1,Argentina,ARG,Latin America,2018,Grassland,17300.000,81.650336,4.719673,NaN,NaN,NaN,"Ludemann, C.I., Gruere, A., Heffer, P. et al. ...",NaN
2,Argentina,ARG,Latin America,2018,Maize,9039.594,439.956488,48.669939,NaN,NaN,NaN,"Ludemann, C.I., Gruere, A., Heffer, P. et al. ...",NaN
3,Argentina,ARG,Latin America,2018,Other cereals,1875.349,86.773543,46.270610,NaN,NaN,NaN,"Ludemann, C.I., Gruere, A., Heffer, P. et al. ...",NaN
4,Argentina,ARG,Latin America,2018,Residual,4947.668,51.220470,10.352447,NaN,NaN,NaN,"Ludemann, C.I., Gruere, A., Heffer, P. et al. ...",NaN


In [48]:
# read in emission factors

emission_factors = pd.read_csv('..\data\\emission-factors.csv', encoding = 'latin-1')

emission_factors

# show the row of the country "global"


,COD,Agri-Footprint Region,Country,IPCC Climate Koppen,DE EF,DE EF Level,IE Volitisation Factor IPCC,IE EF Volatilization and redeposition,IE Leaching Factor IPCC,IE EF Leaching and runoff,EF urea IPCC,EF transport (global) (kgN/ kgCO2),EF production (global) (kgN/ kgCO2)
0,1,Middle East,Afghanistan,dry,0.009,Global,0.11,0.005,0.00,0.011,0.2,0.277,3.73
1,2,Europe,Albania,wet,0.007,Regional,0.11,0.014,0.24,0.011,0.2,0.277,3.73
2,3,Africa,Algeria,dry,0.003,Regional,0.11,0.005,0.00,0.011,0.2,0.277,3.73
3,4,Oceania,American_Samoa,wet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.277,3.73
4,5,Europe,Andorra,wet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.277,3.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,228,Oceania,Wallis and Futuna Islands,wet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.277,3.73
228,229,Africa,Western Sahara,dry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.277,3.73
229,230,Middle East,Yemen,dry,-0.032,Global,0.11,0.005,0.00,0.011,0.2,0.277,3.73
230,231,Africa,Zambia,wet,-0.032,Regional,0.11,0.014,0.24,0.011,0.2,0.277,3.73


In [49]:
# create ISO3 code for each country in the emission factors dataset
iso_codes = pd.read_csv('..\data-clean\\country_iso3.csv')


# create a new column in the emission factors dataset with the ISO3 code
emission_factors['ISO3'] = emission_factors['Country'].map(iso_codes.set_index('country')['iso3'])

emission_factors.head()

# print the countries without an ISO3 code
emission_factors[emission_factors['ISO3'].isnull()]['Country'].unique()

# remove every country that includes china except "china,_mainland"
countries_to_remove = ['China', 'China,_Hong_Kong_SAR', 'China,_Macao_SAR', 'China,_Taiwan_Province_of']
emission_factors = emission_factors[~emission_factors['Country'].isin(countries_to_remove)]

# rename the country "china,_mainland" to "china"
emission_factors['Country'] = emission_factors['Country'].replace('China,_mainland', 'China')


C:\Users\kraos\AppData\Local\Temp\ipykernel_26960\284112757.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emission_factors['Country'] = emission_factors['Country'].replace('China,_mainland', 'China')


In [51]:
# create a new dataframe with the columns "crop", "country", "iso3" "direct emissions", "indirect emissions", "production emissions", "transport emissions", "urea emissions")
crop_country_emissions = crop_country[['crop', 'country', 'iso3_code','aver_n_rate_kg_ha']]

crop_country_emissions = crop_country_emissions.merge(emission_factors, left_on = 'iso3_code', right_on = 'ISO3', how = 'left')

# replace the negative values with 0.05 (which does not really make sense why there would be any negative values, so I resorted to the IPCC default for dry countries (which all of them are and which is 0.05))
crop_country_emissions.loc[crop_country_emissions['DE EF'] < 0, 'DE EF'] = 0.05

# create the column "direct emissions" with the following formula: crop_country["aver_n_rate_kg_ha"] * emission_factor["DE EF"] * 44/28 * 265 where the emission factor is the one for the country (if available) or the global one
crop_country_emissions['direct_emissions'] = crop_country_emissions['aver_n_rate_kg_ha'] * crop_country_emissions['DE EF'] * 44/28 * 265

crop_country_emissions['indirect_emissions_VR'] = crop_country_emissions['aver_n_rate_kg_ha'] * crop_country_emissions['IE EF Volatilization and redeposition'] * 44/28 * 265

crop_country_emissions['indirect_emissions_LR'] = crop_country_emissions['aver_n_rate_kg_ha'] * crop_country_emissions['IE EF Leaching and runoff'] * 44/28 * 265

crop_country_emissions['indirect_emissions_total'] = crop_country_emissions['indirect_emissions_VR'] + crop_country_emissions['indirect_emissions_LR']

crop_country_emissions['production_emissions'] = crop_country_emissions['aver_n_rate_kg_ha'] * crop_country_emissions['EF production (global) (kgN/ kgCO2)']

crop_country_emissions['transport_emissions'] = crop_country_emissions['aver_n_rate_kg_ha'] * crop_country_emissions['EF transport (global) (kgN/ kgCO2)']

crop_country_emissions['urea emissions'] = crop_country_emissions['aver_n_rate_kg_ha'] * crop_country_emissions['EF urea IPCC'] *44/12

# create a column "total emissions" with the sum of all emissions
crop_country_emissions['total_emissions'] = crop_country_emissions['direct_emissions'] + crop_country_emissions['indirect_emissions_total'] + crop_country_emissions['production_emissions'] + crop_country_emissions['transport_emissions'] + crop_country_emissions['urea emissions']



# print the shrae of nan values for each  column
crop_country_emissions.isnull().sum()/len(crop_country_emissions)

crop_country_emissions


# show all the emissions that are negative
crop_country_emissions[crop_country_emissions['total_emissions'] < 0]

crop_country_emissions







,crop,country,iso3_code,aver_n_rate_kg_ha,COD,Agri-Footprint Region,Country,IPCC Climate Koppen,DE EF,DE EF Level,...,EF production (global) (kgN/ kgCO2),ISO3,direct_emissions,indirect_emissions_VR,indirect_emissions_LR,indirect_emissions_total,production_emissions,transport_emissions,urea emissions,total_emissions
0,Fruit primary and citrus fruits and treenuts,Argentina,ARG,53.516528,9,Latin America,Argentina,dry,0.024,Regional,...,3.73,ARG,534.859475,111.429057,245.143926,356.572983,199.616651,14.824078,39.245454,1145.118642
1,Grassland,Argentina,ARG,4.719673,9,Latin America,Argentina,dry,0.024,Regional,...,3.73,ARG,47.169756,9.827033,21.619472,31.446504,17.604379,1.307349,3.461093,100.989082
2,Maize,Argentina,ARG,48.669939,9,Latin America,Argentina,dry,0.024,Regional,...,3.73,ARG,486.421276,101.337766,222.943085,324.280851,181.538872,13.481573,35.691289,1041.413861
3,Other cereals,Argentina,ARG,46.270610,9,Latin America,Argentina,dry,0.024,Regional,...,3.73,ARG,462.441701,96.342021,211.952446,308.294468,172.589377,12.816959,33.931781,990.074286
4,Residual,Argentina,ARG,10.352447,9,Latin America,Argentina,dry,0.024,Regional,...,3.73,ARG,103.465312,21.555273,47.421601,68.976875,38.614627,2.867628,7.591794,221.516236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,Rice,Viet Nam,VNM,127.615866,227,East Asia,Viet Nam,wet,0.050,Regional,...,3.73,VNM,2657.144641,744.000499,584.571821,1328.572320,476.007181,35.349595,93.584968,4590.658705
749,Roots/tubers,Viet Nam,VNM,47.088614,227,East Asia,Viet Nam,wet,0.050,Regional,...,3.73,VNM,980.452204,274.526617,215.699485,490.226102,175.640529,13.043546,34.531650,1693.894031
750,Soybeans,Viet Nam,VNM,53.212015,227,East Asia,Viet Nam,wet,0.050,Regional,...,3.73,VNM,1107.950173,310.226048,243.749038,553.975086,198.480817,14.739728,39.022144,1914.167948
751,Sugar cane,Viet Nam,VNM,218.056339,227,East Asia,Viet Nam,wet,0.050,Regional,...,3.73,VNM,4540.244487,1271.268456,998.853787,2270.122243,813.350144,60.401606,159.907982,7844.026462


In [52]:
# export the dataframe to a csv file
crop_country_emissions.to_csv('..\data-final\\N_use_crop_country_emissions.csv', index = False)